# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-05 17:43:48] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-05 17:43:48] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-05 17:43:48] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-05 17:43:51] INFO server_args.py:1615: Attention backend not specified. Use fa3 backend by default.


[2026-01-05 17:43:51] INFO server_args.py:2502: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=56 avail_mem=59.33 GB):  45%|████▌     | 9/20 [00:00<00:00, 16.71it/s]

Capturing batches (bs=24 avail_mem=59.31 GB):  65%|██████▌   | 13/20 [00:00<00:00, 17.32it/s]

Capturing batches (bs=4 avail_mem=59.29 GB):  75%|███████▌  | 15/20 [00:01<00:00, 15.94it/s]

Capturing batches (bs=1 avail_mem=59.28 GB):  90%|█████████ | 18/20 [00:01<00:00, 16.58it/s]

Capturing batches (bs=1 avail_mem=59.28 GB): 100%|██████████| 20/20 [00:01<00:00, 14.10it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Adam Kornak. I am a data scientist, AI researcher, and software engineer at NVIDIA. I have been working on AI and machine learning for over 15 years. I'm part of the Machine Learning Research Group at NVIDIA, where I am a member of the NVIDIA AI Research team. I joined NVIDIA in 2015 as a Senior Software Engineer. Prior to joining NVIDIA, I had a long career at Microsoft, working on AI research at Microsoft Research in Redmond, Washington, and Microsoft Research Cambridge, United Kingdom, where I led the AI team. Before that, I worked for Google, Google Research, and
Prompt: The president of the United States is
Generated text:  36 years older than the president of Peru. The president of Peru is taller than the president of Brazil. If the president of Brazil is 18 years old, how old is the president of Peru?
Let's break down the information given:

1. The president of Brazil is 18 years old.
2. The president of Peru is taller than the presiden

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I am a [occupation] who has been working in the [industry] for [number] years. I am passionate about [reason for interest] and I am always looking for ways to [action or goal]. I am a [character trait] and I am always [positive or negative] about [thing that makes me happy]. I am [character description] and I am always [positive or negative] about [thing that makes me happy]. I am [character trait] and I am always [positive or negative] about [thing that makes me happy]. I am [character description] and I am always [

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also a major cultural and economic center, hosting numerous museums, theaters, and other cultural institutions. Paris is a popular tourist destination and a major hub for international business and diplomacy. It is also home to the French Parliament and the French National Assembly. The city is known for its rich history, including the influence of the French Revolution and the influence of the French Revolution on the arts and sciences. Paris is a city of contrasts, with its modern architecture and historical landmarks blending seamlessly into one another.

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. These technologies are expected to continue to improve and become more integrated into our daily lives, from self-driving cars to personalized healthcare and financial services. Additionally, AI is likely to continue to be used for ethical and social purposes, such as improving access to education and healthcare for marginalized communities. Finally, AI is likely to continue to evolve and adapt to new challenges and opportunities, as we continue to explore and harness the full potential of this transformative technology.



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name]. I'm a [job title] who loves to explore new things and always looks to expand my horizons. I'm passionate about learning and trying new things, and I'm always looking for ways to grow and improve. I'm a team player and love helping others find their own path. I'm here to introduce myself. And I'll be happy to tell you more about my interests, experiences, and goals. [Name] [Location] [Work/Study/Other] Here's your introduction, [Name]. Please feel free to ask me any questions you may have! 🌍✨🎉🔍✨🌍 #

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the city that offers a rich tapestry of cultural and artistic experiences, world-renowned fashion, and a vibrant nightlife scene.

What would be the answer to this grammatically incorrect sentence: "Paris is the capital of France"? The sentence is gramma

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

:

 AI

 is

 expected

 to

In [6]:
llm.shutdown()